In [1]:
using Revise

In [2]:
using Flux
using Plots; gr()
using JLD2, FileIO

In [3]:
import LearningMPC

In [222]:
samples = load("2017-12-20-hopper-grid/grid_search.jld2")["samples"];

filter!(samples) do sample
    (sample.state[1] == 0.25 &&
     sample.state[2] == 0.25 &&
     sample.state[3] == sample.state[4])
end

# samples = samples[1:100:length(samples)]

x = [[s.state[3]] for s in samples]
y = [[(s.mip.objective_bound, s.mip.objective_value)] for s in samples];
# training_data = collect(Iterators.repeated(collect(zip(x, y)), 10))
training_data = collect(zip(x, y))
shuffle!(training_data)

51-element Array{Tuple{Array{Float64,1},Array{Tuple{Float64,Float64},1}},1}:
 ([1.8], Tuple{Float64,Float64}[(131.363, 134.297)]) 
 ([4.6], Tuple{Float64,Float64}[(88.4009, 96.5646)]) 
 ([2.8], Tuple{Float64,Float64}[(107.194, 117.403)]) 
 ([0.8], Tuple{Float64,Float64}[(135.88, 141.653)])  
 ([5.0], Tuple{Float64,Float64}[(95.2335, 104.025)]) 
 ([-3.8], Tuple{Float64,Float64}[(193.278, 208.502)])
 ([3.4], Tuple{Float64,Float64}[(111.905, 116.826)]) 
 ([-0.2], Tuple{Float64,Float64}[(150.385, 157.912)])
 ([-1.0], Tuple{Float64,Float64}[(167.776, 182.471)])
 ([-1.4], Tuple{Float64,Float64}[(172.148, 188.321)])
 ([-0.6], Tuple{Float64,Float64}[(156.636, 164.159)])
 ([0.0], Tuple{Float64,Float64}[(144.468, 156.247)]) 
 ([-2.6], Tuple{Float64,Float64}[(179.341, 194.565)])
 ⋮                                                   
 ([0.6], Tuple{Float64,Float64}[(150.057, 150.835)]) 
 ([3.6], Tuple{Float64,Float64}[(114.932, 116.698)]) 
 ([-3.0], Tuple{Float64,Float64}[(182.887, 198.112)])
 ([2.

In [231]:
function build_model()
    net = Flux.Chain(
        Flux.Dense(1, 10, Flux.elu),
        Flux.Dense(10, 10, Flux.elu),
        Flux.Dense(10, 1, Flux.elu))

    loss = (x, y) -> begin
#         norm(y[1][1] - net(x))
        @assert length(y) == 1
#         sum((net(x) .- y[1][1]).^2)
#         abs(net(x)[] - y[1][1])
        lower, upper = y[1]
        ŷ = net(x)[]
        if ŷ < lower
            (lower - ŷ)
        elseif ŷ > upper
            (ŷ - upper)
        else
            0 * ŷ
        end
    end
    net, loss
end

net, loss = build_model()

# opt = Flux.Optimise.Momentum(Flux.params(net), 1e-4)
opt = ADAM(params(net))

(::#71) (generic function with 1 method)

In [232]:
plt = scatter(first.(first.(training_data)), first.(first.(last.(training_data))))
scatter!(plt, first.(first.(training_data)), last.(first.(last.(training_data))), markercolor=colorant"red")
xs = linspace(-5, 5, 100)
plot!(plt, xs, first.(Flux.Tracker.value.(net.(xs))))
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 150 
 
 
 200 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip3702)" style="stroke:#3da44d; stroke-width:1; stroke-opacity:1; fill:none" points="
 50.9357,365.223 56.1273,365.229 61.3188,365.236 66.5103,365.242 71.7018,365.248 76.8933,365.254 82.0848,365.259 87.2764,365.265 92.4679,365.27 97.6594,365.275 
 102.851,365.28 108.042,365.284 113.234,365.289 118.425,365.293 123.617,365.298 128.809,365.302 134,365.306 139.192,365.31 144.383,365.315 149.575,365.319 
 154.766,365.324 159.958,365.329 165.149,365.334 170.341,365.339 175.532,365.345 180.724,365.351 185.915,365.357 191.107,365.364 196.298,365.37 201.49,365.377 
 206.681,365.385 211.873,365.392 217.064,365.4 222.256,365.408 227.447,365.416 232.639,365.424 237.83,365.433 243.022,365.442 248.213,365.451 253.405,365.459 
 258.597,365.468 263.788,365.476 268.98,365.484 274.171,365.492 279.363,365.499 284.554,365.504 289.746,365.509 294.937,365.511 300.129,365.511 305.32,365.509 
 310.512,365.503 315.703,365.497 320.895,365.49 326.086,365.483 331.278,365.476 336.469,365.468 341.661,365.461 346.852,365.453 352.044,365.444 357.235,365.436 
 362.427,365.427 367.618,365.418 372.81,365.409 378.001,365.4 383.193,365.39 388.384,365.38 393.576,365.37 398.768,365.36 403.959,365.349 409.151,365.338 
 414.342,365.327 419.534,365.315 424.725,365.303 429.917,365.291 435.108,365.278 440.3,365.265 445.491,365.252 450.683,365.238 455.874,365.224 461.066,365.21 
 466.257,365.195 471.449,365.18 476.64,365.165 481.832,365.15 487.023,365.134 492.215,365.118 497.406,365.102 502.598,365.085 507.789,365.069 512.981,365.052 
 518.172,365.034 523.364,365.017 528.556,364.999 533.747,364.981 538.939,364.963 544.13,364.944 549.322,364.926 554.513,364.907 559.705,364.888 564.896,364.869 
 
 "/>
 
 
 
 
 
 y1 
 
 
 
 
 y2 
 
 
 
 y3

In [233]:
for i in 1:100
    Flux.train!(loss, training_data, opt)
    shuffle!(training_data)
end

In [234]:
loss(training_data[1]...)

Tracked 0-dimensional Array{Float64,0}:
0.632088

In [235]:
plt = scatter(first.(first.(training_data)), first.(first.(last.(training_data))))
scatter!(plt, first.(first.(training_data)), last.(first.(last.(training_data))), markercolor=colorant"red")
xs = linspace(-5, 5, 100)
plot!(plt, xs, first.(Flux.Tracker.value.(net.(xs))))
plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 100 
 
 
 120 
 
 
 140 
 
 
 160 
 
 
 180 
 
 
 200 
 
 
 220 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip3902)" style="stroke:#3da44d; stroke-width:1; stroke-opacity:1; fill:none" points="
 50.9357,22.946 56.1273,28.4825 61.3188,33.9041 66.5103,39.2109 71.7018,44.4153 76.8933,49.5103 82.0848,54.4883 87.2764,59.3808 92.4679,64.2418 97.6594,69.068 
 102.851,73.8553 108.042,78.5984 113.234,83.2915 118.425,87.9275 123.617,92.498 128.809,96.9926 134,101.399 139.192,105.701 144.383,109.88 149.575,113.953 
 154.766,117.946 159.958,121.85 165.149,125.657 170.341,129.359 175.532,132.945 180.724,136.405 185.915,139.725 191.107,142.893 196.298,145.894 201.49,148.712 
 206.681,151.348 211.873,153.951 217.064,156.563 222.256,159.183 227.447,161.813 232.639,164.452 237.83,167.101 243.022,169.762 248.213,172.435 253.405,175.12 
 258.597,177.818 263.788,180.531 268.98,183.258 274.171,186.002 279.363,188.761 284.554,191.54 289.746,194.353 294.937,197.208 300.129,200.107 305.32,203.055 
 310.512,206.057 315.703,209.116 320.895,212.239 326.086,215.432 331.278,218.699 336.469,222.049 341.661,225.488 346.852,229.025 352.044,232.669 357.235,236.414 
 362.427,240.213 367.618,244.004 372.81,247.785 378.001,251.404 383.193,254.882 388.384,258.273 393.576,261.63 398.768,265.003 403.959,268.439 409.151,271.984 
 414.342,275.638 419.534,279.198 424.725,282.645 429.917,285.993 435.108,289.254 440.3,292.439 445.491,295.557 450.683,298.615 455.874,301.621 461.066,304.582 
 466.257,307.487 471.449,310.254 476.64,312.883 481.832,315.379 487.023,317.667 492.215,319.737 497.406,321.604 502.598,323.282 507.789,324.785 512.981,326.125 
 518.172,327.315 523.364,328.363 528.556,329.281 533.747,330.077 538.939,330.76 544.13,331.338 549.322,331.818 554.513,332.207 559.705,332.511 564.896,332.706 
 
 "/>
 
 
 
 
 
 y1 
 
 
 
 
 y2 
 
 
 
 y3